In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

C:\Users\billt\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#helper functions
def get_train_and_test_splits(train_size, batch_size=1):
    # We prefetch with a buffer the same size as the dataset because th dataset
    # is very small and fits into memory.
    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
        .map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=dataset_size)
        .cache()
    )
    # We shuffle with a buffer the same size as the dataset.
    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset

In [3]:
hidden_units = [8, 8]
learning_rate = 0.001


def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")

In [4]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs

In [10]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

In [11]:
def create_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [12]:
num_epochs = 500
train_sample_size = int(train_size * 0.3)
small_train_dataset = train_dataset.unbatch().take(train_sample_size).batch(batch_size)

bnn_model_small = create_bnn_model(train_sample_size)
run_experiment(bnn_model_small, mse_loss, small_train_dataset, test_dataset)

Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


Start training the model...
Epoch 1/500
5/5 [==============================] - 3s 395ms/step - loss: 26.4629 - root_mean_squared_error: 5.1427 - val_loss: 29.7204 - val_root_mean_squared_error: 5.4506
Epoch 2/500
5/5 [==============================] - 0s 7ms/step - loss: 27.5263 - root_mean_squared_error: 5.2453 - val_loss: 26.1698 - val_root_mean_squared_error: 5.1148
Epoch 3/500
5/5 [==============================] - 0s 7ms/step - loss: 25.3660 - root_mean_squared_error: 5.0349 - val_loss: 23.3549 - val_root_mean_squared_error: 4.8312
Epoch 4/500
5/5 [==============================] - 0s 8ms/step - loss: 27.9760 - root_mean_squared_error: 5.2879 - val_loss: 24.9474 - val_root_mean_squared_error: 4.9934
Epoch 5/500
5/5 [==============================] - 0s 7ms/step - loss: 24.7464 - root_mean_squared_error: 4.9730 - val_loss: 27.1635 - val_root_mean_squared_error: 5.2105
Epoch 6/500
5/5 [==============================] - 0s 7ms/step - loss: 25.1009 - root_mean_squared_error: 5.0088 - 

In [13]:
def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(examples).numpy())
    predicted = np.concatenate(predicted, axis=1)

    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

    for idx in range(sample):
        print(
            f"Predictions mean: {round(prediction_mean[idx], 2)}, "
            f"min: {round(prediction_min[idx], 2)}, "
            f"max: {round(prediction_max[idx], 2)}, "
            f"range: {round(prediction_range[idx], 2)} - "
            f"Actual: {targets[idx]}"
        )


compute_predictions(bnn_model_small)

Predictions mean: 5.26, min: 4.27, max: 5.97, range: 1.7 - Actual: 5.0
Predictions mean: 5.16, min: 4.37, max: 5.95, range: 1.58 - Actual: 5.0
Predictions mean: 5.6, min: 4.66, max: 6.24, range: 1.58 - Actual: 6.0
Predictions mean: 5.18, min: 4.29, max: 5.81, range: 1.52 - Actual: 5.0
Predictions mean: 6.33, min: 5.88, max: 6.5, range: 0.61 - Actual: 7.0
Predictions mean: 5.69, min: 4.89, max: 6.25, range: 1.36 - Actual: 5.0
Predictions mean: 6.08, min: 5.73, max: 6.37, range: 0.64 - Actual: 6.0
Predictions mean: 6.28, min: 5.92, max: 6.48, range: 0.56 - Actual: 6.0
Predictions mean: 5.24, min: 4.52, max: 6.07, range: 1.55 - Actual: 4.0
Predictions mean: 6.0, min: 5.31, max: 6.39, range: 1.09 - Actual: 5.0


In [14]:
num_epochs = 500
bnn_model_full = create_bnn_model(train_size)
run_experiment(bnn_model_full, mse_loss, train_dataset, test_dataset)

compute_predictions(bnn_model_full)

Start training the model...
Epoch 1/500
17/17 [==============================] - 2s 25ms/step - loss: 29.6806 - root_mean_squared_error: 5.4476 - val_loss: 31.0183 - val_root_mean_squared_error: 5.5690
Epoch 2/500
17/17 [==============================] - 0s 3ms/step - loss: 31.9192 - root_mean_squared_error: 5.6493 - val_loss: 32.1312 - val_root_mean_squared_error: 5.6681
Epoch 3/500
17/17 [==============================] - 0s 3ms/step - loss: 28.8203 - root_mean_squared_error: 5.3680 - val_loss: 30.3278 - val_root_mean_squared_error: 5.5066
Epoch 4/500
17/17 [==============================] - 0s 3ms/step - loss: 26.7605 - root_mean_squared_error: 5.1726 - val_loss: 26.6763 - val_root_mean_squared_error: 5.1644
Epoch 5/500
17/17 [==============================] - 0s 3ms/step - loss: 28.2726 - root_mean_squared_error: 5.3168 - val_loss: 27.3638 - val_root_mean_squared_error: 5.2306
Epoch 6/500
17/17 [==============================] - 0s 3ms/step - loss: 25.6528 - root_mean_squared_error